In [ ]:
%reload_ext autoreload
%autoreload 2
import collections
import numpy
import os
import sklearn.preprocessing
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from classifier import classification_data

In [ ]:
cfg = {
    "fold": 5,
    "n_labels": 5,
    "path_base": "dataset",
    "path_out": "out",
    "test_size": 0.2,
    "train_size": 0.8,
    "random_state": 1
}

In [ ]:
surf = numpy.loadtxt(os.path.join(cfg["path_base"], "surf64.txt"))
dataset = "surf"
n_samples, n_features = surf.shape
n_features = n_features - 1
x, y = surf[0:, 0:n_features], surf[:, n_features]
x_normalized = sklearn.preprocessing.StandardScaler().fit_transform(x)

print(x.shape, y.shape)

In [ ]:
kf = sklearn.model_selection.KFold(n_splits=cfg["fold"], shuffle=True, random_state=cfg["random_state"])
l = list([])
for (train_index, test_index) in kf.split(x):
    l = l + test_index.tolist()
print(len(list(set(l))))
print(collections.Counter(sorted(l)))

In [ ]:
for pca in (128, 256, -1):
    x = x_normalized
    if pca > 0:
        x = sklearn.decomposition.PCA(n_components=pca).fit_transform(x)

    if pca == -1:
        pca = n_features

    classification_data(cfg, dataset, list(kf.split(x)), pca, n_samples, x, y)
    break